## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


In [ ]:
query = "SELECT DISTINCT(task) FROM dlc_table;"
df = pd.read_sql_query(query, conn)

print(df)

## Import plot_feature_barplot function 

In [ ]:
import Python_scripts.Data_analysis.plot_feature_barplot
import Python_scripts.Data_analysis.plot_feature_histogram_and_boxplot

importlib.reload(Python_scripts.Data_analysis.plot_feature_barplot)
importlib.reload(Python_scripts.Data_analysis.plot_feature_histogram_and_boxplot)

from Python_scripts.Data_analysis.plot_feature_barplot import plot_feature_barplot
from Python_scripts.Data_analysis.plot_feature_histogram_and_boxplot import plot_feature_histogram_and_boxplot


## Feature barplot

In [ ]:
plot_feature_histogram_and_boxplot(conn, 'mean_curv')

## Feature distribution

In [ ]:
import Python_scripts.Data_analysis.plot_feature_distribution
importlib.reload(Python_scripts.Data_analysis.plot_feature_distribution)
from Python_scripts.Data_analysis.plot_feature_distribution import plot_feature_distribution


In [ ]:
print(type(FoodOnly_saline_id), type(FoodOnly_saline_id[0]))
print(type(FoodOnly_ghrelin_id), type(FoodOnly_ghrelin_id[0]))


In [ ]:
ax = plot_feature_distribution(conn, FoodOnly_saline_id, FoodOnly_ghrelin_id, feature='mean_curv', 
                              group_labels=['Saline', 'Ghrelin'], mode='pdf')

ax.set_title('Food Only')
plt.show()


## Grid plot for feature summary

In [ ]:
from Python_scripts.Utility_functions import feature_summary

fig, ax = feature_summary.plot_feature_grid(
    n_tasks=7,
    n_features=4,
    task_labels=["Combined", "NoFoodOnly", "FoodOnly", "ToyOnly", "LightOnly", "FoodLight", "ToyLight"],
    feature_labels=["Curvature", "Distance", "Angle", "Stops"],
    show=True
)

fig.savefig("summary_grid.pdf", dpi=300)


In [ ]:
from Python_scripts.Utility_functions import feature_summary_dual as fsd

fig, ax = fsd.plot_dual_comparison_grid(
    n_tasks=7,
    n_features=4,
    task_labels=["Combined", "NoFoodOnly", "FoodOnly", "ToyOnly", "LightOnly", "FoodLight", "ToyLight"],
    feature_labels=["Curvature","Distance","Angle","Stops"],
    show=False
)
fig.savefig("dual_grid_vector.pdf", dpi=300, bbox_inches="tight")
